# Bloch-Torrey Equation

## Introduction

Here we solve the Bloch-Torrey equation on a unit square, with the diffusion coefficient $D(x)$, relaxation rates $R_1(x)$ and $R_2(x)$, and resonance frequency $\omega(x)$ all given as a generic functions.
The strong form of the Bloch-Torrey equation is given by

\begin{align}
\begin{cases}
    \frac{\partial M_x}{\partial t} &= \nabla \cdot (D \cdot \nabla M_x) - R_2 \, M_x + \omega \, M_y\\
    \frac{\partial M_y}{\partial t} &= \nabla \cdot (D \cdot \nabla M_y) - R_2 \, M_y - \omega \, M_x\\
    \frac{\partial M_z}{\partial t} &= \nabla \cdot (D \cdot \nabla M_z) - R_1 \, (M_z - M_{z,\infty}),
\end{cases}\label{eq:strong_M}\tag{0}
\end{align}

where $\vec{M}(x,t) = [M_x(x,t), \, M_y(x,t), \, M_z(x,t)]$ is the magnetization vector.

Letting $\vec{u}(x,t) = [M_x(x,t), \, M_y(x,t), \, M_{z,\infty} - M_z(x,t)]$, equation (\ref{eq:strong_M}) becomes

\begin{align}
\begin{cases}
    \frac{\partial u_x}{\partial t} &= \nabla \cdot (D \cdot \nabla u_x) - R_2 \, u_x + \omega \, u_y\\
    \frac{\partial u_y}{\partial t} &= \nabla \cdot (D \cdot \nabla u_y) - R_2 \, u_y - \omega \, u_x\\
    \frac{\partial u_z}{\partial t} &= \nabla \cdot (D \cdot \nabla u_z) - R_1 \, u_z,
\end{cases}\label{eq:strong}\tag{1}
\end{align}

Equation (\ref{eq:strong}) holds for all points $x$ in the domain $\Omega$.

Note: $\cdot$ is generally a tensor contraction, unless the arguments are scalar, in which it reduces to multiplication.

We consider homogeneous Neumann boundary conditions such that

\begin{align}
    D \, \nabla \vec{u}(x) \cdot \hat{n} &= \vec{0}  \quad \forall x \in \partial \Omega\label{eq:bc}\tag{2}\\
\end{align}

where $\partial \Omega$ denotes the boundary of $\Omega$. The initial condition is assumed to be specified,

\begin{equation}
    \vec{u}(x,t=0) = \vec{u}_0 (x)  \quad \forall x \in \Omega.\label{eq:ic}\tag{3}
\end{equation}

The weak form corresponding to equation \ref{eq:strong} is given by

\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    &= -\int_{\Omega}
    -\vec{v} \cdot \nabla \cdot ( D \, \nabla \vec{u} ) +
    \vec{v} \cdot (\vec{R} \odot \vec{u}) -
    \omega \, \vec{v}_\perp \times \vec{u}_\perp
    \, d\Omega \\
    &= -\int_{\Omega}
    \nabla \vec{v} : D \, \nabla \vec{u} +
    \vec{v} \cdot (\vec{R} \odot \vec{u}) -
    \omega \, \vec{v}_\perp \times \vec{u}_\perp
    \, d\Omega + 
    \int_{\partial\Omega} \vec{v} \cdot (D \, \nabla\vec{u} \cdot \hat{n}) \, d\Gamma,
\end{align}

where $\vec{v}$ is a suitable test function, $\vec{R} = [R_2, R_2, R_1]$, $\odot$ is the Hadamard (elementwise) product, and $\vec{u}_\perp$ is the transverse component $[u_x, u_y]$ of $\vec{u}$.

Applying the homogeneous Neumann boundary conditions from equation (\ref{eq:bc}), the final weak form becomes
\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    = -\int_{\Omega}
    \nabla \vec{v} : D \, \nabla \vec{u} +
    \vec{v} \cdot (\vec{R} \odot \vec{u}) -
    \omega \, \vec{v}_\perp \times \vec{u}_\perp
    \, d\Omega.\label{eq:weak}\tag{4}
\end{align}

Note: in two dimensions, the cross product is simply a scalar. However, `Tensors.jl` defines the two dimensional cross product by first extending the 2D vectors into 3D. Below, we use the symbol $\boxtimes$ to denote the scalar version, which is the same as taking the third component of the vector version

In [1]:
# Activate project
import Pkg
Pkg.activate(joinpath(@__DIR__, ".."))
include(joinpath(@__DIR__, "../initpaths.jl"))

In [2]:
using StatsPlots, BSON, Dates
pyplot(size=(1200,900)); # NOTE: must load pyplot backend BEFORE loading MATLAB in MWFUtils

In [3]:
using MWFUtils
# mxcall(:cd, 0, pwd()) # change MATLAB path to current path for saving outputs
# mxcall(:figure, 0) # bring up MATLAB figure gui

**Load Geometry**

In [4]:
geomfile = joinpath(
    "/home/jdoucette/Documents/code/BlochTorreyResults/Experiments/MyelinWaterOrientation/Geometries/kmg_geom_sweep_3",
    "2019-03-28-T-15-24-11-877__N-10_g-0.7500_p-0.7500__structs.bson" # 1.3k triangles, 1.2k points, Qmin = 0.3
    # "2019-03-28-T-15-26-44-544__N-10_g-0.8000_p-0.8300__structs.bson" # 4.7k triangles, 3.2k points, Qmin = 0.3
    # "2019-03-28-T-15-27-56-042__N-20_g-0.7500_p-0.7000__structs.bson" # 3.1k triangles, 2.6k points, Qmin = 0.3
    # "2019-03-28-T-15-33-59-628__N-20_g-0.8000_p-0.8000__structs.bson" #13.3k triangles, 9.2k points, Qmin = 0.3
)
geom = loadgeometry(geomfile);

In [5]:
exteriorgrids, torigrids, interiorgrids, outercircles, innercircles, bdry = geom;
ferritins = Vec{3,floattype(bdry)}[];

**Set parameters**

In [245]:
btparams = BlochTorreyParameters{Float64}(
    B0 = 0.0, # for testing
    theta = π/2, # [radians]
    D_Tissue = 50.0, # [μm²/s]
    D_Sheath = 50.0, # [μm²/s]
    D_Axon = 50.0, # [μm²/s]
    K_perm = 0.5, # [μm/s]
    );

**Create domains**

In [273]:
# utype = Vec{3,Float64};
# utype = Vec{2,Float64};
utype = ComplexF64;

In [274]:
myelinprob = MyelinProblem(btparams);

In [275]:
myelinsubdomains = createmyelindomains(
    vec(exteriorgrids), vec(torigrids), vec(interiorgrids),
    vec(outercircles), vec(innercircles), vec(ferritins),
    utype);

In [276]:
myelinprob, myelinsubdomains, myelindomains = createdomains(btparams,
    exteriorgrids, torigrids, interiorgrids,
    outercircles, innercircles, ferritins, utype);

    Assemble subdomains    

┌ Info: Assembling MyelinDomain from subdomains
└ @ MWFUtils /home/jdoucette/Documents/code/BlochTorreyExperiments-master/MyelinWaterTools/MWFUtils/src/mwfutils.jl:321


 0.720280 seconds (935.66 k allocations: 46.175 MiB, 2.71% gc time)
    Factorize subdomains    0.050490 seconds (48.89 k allocations: 3.195 MiB)
    Assemble combined       0.306508 seconds (564.19 k allocations: 34.159 MiB, 2.42% gc time)
    Factorize combined      0.004229 seconds (2.81 k allocations: 763.244 KiB)


**Solve Bloch-Torrey equation**

In [280]:
# Solve Bloch-Torrey equation
# u0 = Vec{3}((1.0, 1.0, 1.0));
# u0 = Vec{2}((1.0, 1.0));
u0 = complex(1.0, 1.0);
nTE = 32;
cb = nothing; # for testing
sols = solveblochtorrey(myelinprob, myelindomains; u0 = u0, TE = TE, nTE = nTE, callback = cb);

┌ Info: Solving MyelinProblem
└ @ MWFUtils /home/jdoucette/Documents/code/BlochTorreyExperiments-master/MyelinWaterTools/MWFUtils/src/mwfutils.jl:426


 22.193275 seconds (18.04 M allocations: 2.239 GiB, 4.01% gc time)


# Testing

In [25]:
@show Revise.revision_queue; revise()

Revise.revision_queue = Set(Tuple{Revise.PkgData,String}[])


In [27]:
# utype = ComplexF64; u0 = 1.0 + 1.0im
# utype = Vec2d; u0 = ones(utype)
utype = Vec3d; u0 = ones(utype)
grid = generate_grid(Triangle, (3,3));
domain = ParabolicDomain(grid, utype);
u = interpolate(x->u0, domain);
S = integrate(u, domain)

2-element Tensor{1,2,Float64,2}:
 3.9999999999999614
 3.9999999999999614

In [220]:
# doassemble!(myelinsubdomains[1], myelinprob);
# M = getmass(myelinsubdomains[1]);
# @show M[1:3:end, 1:3:end] |> isposdef;
# @show M[2:3:end, 2:3:end] |> isposdef;
# @show M[3:3:end, 3:3:end] |> isposdef;
# @show M[1:3:end, 1:3:end] ≈ M[2:3:end, 2:3:end];
# @show M[1:3:end, 1:3:end] ≈ M[3:3:end, 3:3:end];
# @show maximum(abs, M[1:3:end, 2:3:end]) ≈ 0;
# @show maximum(abs, M[1:3:end, 3:3:end]) ≈ 0;
# @show maximum(abs, M[2:3:end, 1:3:end]) ≈ 0;
# @show maximum(abs, M[2:3:end, 3:3:end]) ≈ 0;
# @show maximum(abs, M[3:3:end, 1:3:end]) ≈ 0;
# @show maximum(abs, M[3:3:end, 2:3:end]) ≈ 0;